# Import Widget Libraries

In [1]:
import magneton
from magneton.core.idom_loader import reload_bundle
from magneton.core.widget import WidgetModel

magneton_client: 0.0.0


In [2]:
#!pip install --upgrade openai

In [3]:
import os
import json
import openai

# Pre-process Data

In [4]:
import pandas as pd

In [5]:
def transform_data(data_url):
    df = pd.read_csv(data_url)
    df = df[df['Year'] == 2017]
    df = df[['Cohort','State','Drivers']]
    df = df.pivot_table(index=['State'], 
            columns=['Cohort'], values='Drivers').fillna(0)
    df = df.reset_index()
    return df

In [6]:
data_url ='https://data.transportation.gov/api/views/xfkb-3bxx/rows.csv?accessType=DOWNLOAD'

state_data_df = transform_data(data_url)

# Define Actions

In [15]:
def get_data_table(df, option=None, value=None):
    ls = list(df['State'].unique())
    if option == 'prefix':
        ls = [x for x in ls if x.startswith(value)]
    elif option == 'region':
        ls = group_by_region(df, value)
        
    return ls

def group_by_region(df, region):
    states = list(df['State'].unique())
    
    states_str = ", ".join(states)
    
    prompt = "Return comma-separated states in the US {} from the following: {}".format(region, states_str)
    if os.path.exists('../openai_cred.json'):
        openai_cred_file = open('../openai_cred.json')
        openai_cred_json = json.load(openai_cred_file)
        openai_cred_file.close()
        openai.api_key = openai_cred_json['OPENAI_API_KEY']
        openai.organization = openai_cred_json['OPENAI_ORG_KEY']

    completion = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[
            {"role": "user", "content": prompt}
        ]
    )

    result = completion["choices"][0]["message"]["content"]
    states = [s.strip().rstrip() for s in result.split(",")]
    
    return states

In [8]:
def get_distribution_by_state(df, state_name=None):
    result = []
    if state_name:
        row = df[df['State'] == state_name]
        dist = row.loc[:, row.columns != 'State'].to_dict('records')[0]
        for key, value in dist.items():
            result.append({"x": key, "y": value})
    else:
        ls = df.columns
        _df = df.mean(axis=0, numeric_only=True)
        for key in ls:
            if key != 'State':
                result.append({"x": key, "y": _df[key]})
    return result

In [9]:
def get_filtered_distribution_by_buckets(df, state_name=None):
    dist = get_distribution_by_state(df, state_name)
    filtered_ls = []
    for _d in dist:
        if "-" in _d['x']:
            filtered_ls.append(_d)
    return filtered_ls

# Customize All Components and Actions

In [16]:
from magneton import FullyCustomizableLinkedViews
    
def init(): 
    # Fetch initial view data
    return {
        "distribution": get_filtered_distribution_by_buckets(state_data_df),
        "index": -1,
        "table": get_data_table(state_data_df, 'region', 'south')
    }

def select(element):
    # implement selection
    dist = get_filtered_distribution_by_buckets(state_data_df, element)
    index = get_data_table(state_data_df, 'region', 'south').index(element)
    return {
            "distribution": dist,
            "index": index
            }
# Create widget 
lv_widget = FullyCustomizableLinkedViews(state_data_df, 'State', {"init": init, "select": select})

In [17]:
lv_widget.show()

component(119694f40, self=<magneton.core.widget.StatefulWidgetBase.StatefulWidgetBase object at 0x119697340>)

LayoutWidget(Layout(component(119694f40, self=<magneton.core.widget.StatefulWidgetBase.StatefulWidgetBase obje…